<a href="https://colab.research.google.com/github/AmiraOsamaMohamed/GraduationProject/blob/main/NewTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Transformer

In [ ]:
! git clone https://gitlab.com/bigirqu/quranqa.git

fatal: destination path 'quranqa' already exists and is not an empty directory.


**install dependencies**

In [ ]:
! pip install farasa

In [ ]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
! pip install transformers

**import library**

In [ ]:
!ls /content/drive/MyDrive/data

QQA23_TaskB_qrcd_v1.2_dev.csv	       QQA23_TaskB_qrcd_v1.2_test_noAnswers.jsonl  README.md
QQA23_TaskB_qrcd_v1.2_dev.jsonl        QQA23_TaskB_qrcd_v1.2_train.csv		   runs
QQA23_TaskB_qrcd_v1.2_test_gold.csv    QQA23_TaskB_qrcd_v1.2_train.jsonl
QQA23_TaskB_qrcd_v1.2_test_gold.jsonl  quranqa22_eval.cpython-310.pyc


In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import pandas as pd
import numpy as np
import json
import sys
from tqdm import tqdm
import time
sys.path.insert(0,"/content/drive/MyDrive/data/quranqa22_eval.cpython-310.pyc")
import quranqa22_eval as eval #/content/quranqa/code/quranqa22_eval.py

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_train.csv")
validation = pd.read_csv("/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_dev.csv")

In [ ]:
train_dataset_jsonl = []
with open('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_train.jsonl', 'r') as f:
    for line in f:
        train_dataset_jsonl.append(json.loads(line))

with open('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_dev.jsonl', 'r') as f:
    for line in f:
        train_dataset_jsonl.append(json.loads(line))

In [ ]:
len(train_dataset_jsonl)

1155

**prepare data for prediction**

In [ ]:
data = pd.concat([train, validation])

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1155 entries, 0 to 162
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   pq_id               1155 non-null   object
 1   passage             1155 non-null   object
 2   surah               1155 non-null   int64 
 3   verses              1155 non-null   object
 4   question            1155 non-null   object
 5   answer              1155 non-null   object
 6   start_char          1155 non-null   int64 
 7   processed passage   1155 non-null   object
 8   processed question  1155 non-null   object
 9   processed answer    1155 non-null   object
dtypes: int64(2), object(8)
memory usage: 99.3+ KB


In [ ]:
data = data.filter(items=["pq_id","passage","question"])
data_unique_QA = data.drop_duplicates()

In [ ]:
len(data_unique_QA)

1155

**Model**

In [ ]:
from transformers.utils.dummy_pt_objects import RemBertForSequenceClassification
def answer_question(row):
    pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)
    result_row = pipe(question=row["question"], context=row["passage"], handle_impossible_answer=True, topk=5)
    return result_row

In [ ]:
def Try_Transformer(tokenizer_name,model_name ):
  result = {}
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
  model = AutoModelForQuestionAnswering.from_pretrained(model_name)
  # Apply the function to the "question" and "passage" columns of the DataFrame
  for i in tqdm(range(len(data_unique_QA))):
    result_part = answer_question(data_unique_QA.iloc[i])
    result.update({data_unique_QA.iloc[i]["pq_id"]:result_part})
  return result

In [ ]:
all_scores = {}

In [ ]:
tokenizer = "IProject-10/bert-base-uncased-finetuned-squad2"
model = "IProject-10/bert-base-uncased-finetuned-squad2"
result = Try_Transformer(tokenizer, model)
scores = eval.evaluate(train_dataset_jsonl, result)
all_scores.update({tokenizer:scores})
print(scores)

100%|██████████| 1155/1155 [58:26<00:00,  3.04s/it]


{'pRR': 0.05689177348514567, 'exact_match': 0.001221001221001221, 'f1': 0.00681045635270636}
